# Домашнее задание № 2. Мешок слов

In [1]:
%%capture
!pip install nltk razdel pymystem3

In [2]:
import nltk
import numpy as np
import pandas as pd
import random
import razdel

from nltk.corpus import stopwords
from pymystem3 import Mystem
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [3]:
random_seed = 42

np.random.seed(random_seed)
random.seed(random_seed)

In [4]:
nltk.download("stopwords")
russian_stopwords = stopwords.words("russian") + ["весь", "свой", "твой", "это", "очень", "год"]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
mystem = Mystem()

In [6]:
data = pd.read_csv("labeled.csv")
train, test = train_test_split(data, test_size=0.1, shuffle=True, random_state=random_seed)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [7]:
y = train.toxic.values
y_test = test.toxic.values

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя.

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

### TfidfVectorizer с дефолтной токенизацией

In [8]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.3, max_features=100)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

In [9]:
clf = LogisticRegression(C=0.1, class_weight="balanced", random_state=random_seed)
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0, digits=4))

              precision    recall  f1-score   support

         0.0     0.8168    0.7116    0.7606       971
         1.0     0.5302    0.6709    0.5923       471

    accuracy                         0.6983      1442
   macro avg     0.6735    0.6913    0.6765      1442
weighted avg     0.7232    0.6983    0.7056      1442



### TfidfVectorizer + razdel.tokenize

In [10]:
def razdel_tokenize(text):
    # возвращаем список токенов, приведенных к нижнему регистру
    return [token.text.lower() for token in list(razdel.tokenize(text))]

In [11]:
vectorizer = TfidfVectorizer(
    tokenizer=razdel_tokenize,
    token_pattern=None,
    min_df=10,
    max_df=0.3,
    max_features=100,
)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

In [12]:
clf = LogisticRegression(C=0.1, class_weight="balanced", random_state=random_seed)
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0, digits=4))

              precision    recall  f1-score   support

         0.0     0.8194    0.7147    0.7635       971
         1.0     0.5345    0.6752    0.5966       471

    accuracy                         0.7018      1442
   macro avg     0.6769    0.6949    0.6800      1442
weighted avg     0.7263    0.7018    0.7090      1442



Вариант с razdel.tokenize победил: все значения метрик стали выше.


## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [13]:
def mystem_lemmatize(text):
    mystem_lemmata = []
    for token in mystem.analyze(text):
        if "analysis" in token.keys():
            if len(token["analysis"]) > 0:
                mystem_lemmata.append(token["analysis"][0]["lex"])
            else:
                mystem_lemmata.append(token["text"].lower())
    return mystem_lemmata

In [14]:
def setup_experiment(
    vectorizer_type,
    classifier_type,
    vectorizer_kwargs,
    classifier_kwargs,
    print_most_toxic_texts=False,
    print_most_informative_features=False,
):
    match vectorizer_type:
        case "count":
            vectorizer = CountVectorizer(**vectorizer_kwargs)
        case "tfidf":
            vectorizer = TfidfVectorizer(**vectorizer_kwargs)
        case _:
            raise ValueError("Invalid vectorizer type")

    match classifier_type:
        case "logreg":
            classifier = LogisticRegression(**classifier_kwargs)
        case "bayes":
            classifier = MultinomialNB(**classifier_kwargs)
        case "tree":
            classifier = DecisionTreeClassifier(**classifier_kwargs)
        case "forest":
            classifier = RandomForestClassifier(**classifier_kwargs)
        case _:
            raise ValueError("Invalid classifier type")

    classifier.random_state = random_seed
    X = vectorizer.fit_transform(train.comment)
    X_test = vectorizer.transform(test.comment)
    classifier.fit(X, y)
    preds = classifier.predict(X_test)
    print(classification_report(y_test, preds, zero_division=0, digits=4))

    if print_most_toxic_texts:
        probas = classifier.predict_proba(X_test)[:, 1]
        results = pd.DataFrame(
            {"sentence": test.comment, "proba": probas, "true_label": y_test}
        )
        top_n = results.nlargest(10, "proba").reset_index(drop=True)
        print(top_n)
        for i, row in top_n.iterrows():
            print(f"\nText {i}, probability: {row['proba']}")
            print(f"\n{row['sentence']}")

    if print_most_informative_features:
        feature_names = vectorizer.get_feature_names_out()
        match classifier_type:
            case "tree" | "forest":
                importances = classifier.feature_importances_
            case "logreg":
                importances = classifier.coef_[0]
            case "bayes":
                word_vectors = vectorizer.transform(feature_names)
                class_probas = classifier.predict_proba(word_vectors)
                importances = np.squeeze(class_probas[:, 1:], axis=1)
        indices = np.argsort(importances)[::-1][:5]
        top_features = feature_names[indices]
        print("Top 5 toxic words:", ", ".join(top_features))

### TF-IDF + Logistic Regression

In [15]:
setup_experiment(
    "tfidf", "logreg",
    vectorizer_kwargs={
        "tokenizer": mystem_lemmatize,
        "token_pattern": None,
        "stop_words": russian_stopwords,
        "max_df": 0.9,
        "min_df": 0.001,
        "ngram_range": (1, 2)
    },
    classifier_kwargs={
        "C": 0.1,
        "class_weight": "balanced",
    },
    print_most_toxic_texts=True,
)

              precision    recall  f1-score   support

         0.0     0.8963    0.8898    0.8930       971
         1.0     0.7762    0.7877    0.7819       471

    accuracy                         0.8564      1442
   macro avg     0.8362    0.8387    0.8374      1442
weighted avg     0.8570    0.8564    0.8567      1442

                                            sentence     proba  true_label
0             Когда тред прощания с хохлами будет?\n  0.914259         1.0
1  Скрипт триггернулся на сочетание свиньи русски...  0.867450         1.0
2  русня не умеет в готовку ... ведь поджарить хо...  0.861205         1.0
3  Пиздец у быдла с пикабу сначала горело от негр...  0.855859         1.0
4               Шлак для малолетних дебилов и баб.\n  0.852958         1.0
5     Даже хохлы в гейропу не хотят, бабосы не те?\n  0.847751         1.0
6  пук пук пук пук МАЛОЛЕТНИЙ ДЕБИЛ НАСРАЛЬНЕНОК ...  0.843512         1.0
7  Конечно не для русских. Если русский это сразу...  0.836946         1.

### Count + Naive Bayes

In [16]:
setup_experiment(
    "count", "bayes",
    vectorizer_kwargs={
        "tokenizer": mystem_lemmatize,
        "token_pattern": None,
        "stop_words": russian_stopwords,
        "max_df": 0.9,
        "min_df": 0.001,
        "ngram_range": (1, 1)
    },
    classifier_kwargs={
        "alpha": 10.0,
        "fit_prior": False,
    },
    print_most_toxic_texts=True,
)

              precision    recall  f1-score   support

         0.0     0.8874    0.8847    0.8860       971
         1.0     0.7637    0.7686    0.7661       471

    accuracy                         0.8467      1442
   macro avg     0.8256    0.8266    0.8261      1442
weighted avg     0.8470    0.8467    0.8469      1442

                                            sentence     proba  true_label
0  В Киеве на вокзале Мен було рок в 19, коли мен...  1.000000         1.0
1  Ну давай разберём всё тобой написанное. Бляядь...  1.000000         1.0
2  Возьмём как пример Россию, западноевропейские ...  1.000000         0.0
3  моча сюда не заходит И как привлекать их внима...  1.000000         1.0
4  Создал тут тхреад в b 192441781 Как оказалось,...  1.000000         1.0
5  Интересно, а что мы могли сделать? Ввести санк...  1.000000         1.0
6  черт опущенный Гомикадзе Би опущенный гей на е...  1.000000         1.0
7  Пиздец у быдла с пикабу сначала горело от негр...  0.999999         1.

Общий текст только один (топ 4 у TF-IDF + logreg и топ 8 у Count + Bayes).

У каждого эксперимента все топ 10 текстов токсичные, кроме одного.

У TF-IDF + logreg неправильно определенный текст - это топ 10: в нем 4 раза встретилось слово "русский", которое попало в топ 5 токсичных слов у 3 из 4 классификаторов (см. задание 4).

У Count + Bayes "неправильно" определенный текст - это топ 3. Он однозначно токсичный, здесь ошибка разметки.

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов.

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

Извлечение важности признаков - параметр ```print_most_informative_features``` в функции ```setup_experiment```.

Для ```MultinomialNB```: ```classifier.feature_log_prob_``` — это вероятность признака (слова) при условии конкретного класса. Это недостаточно информативно, потому что частые слова будут в топе ```classifier.feature_log_prob_``` у обоих классов. В качестве метрики информативности признака более наглядна вероятность класса при условии наличия конкретного признака (слова). Для этого мы берем все признаки:

```word_vectors = vectorizer.transform(feature_names)```

Считаем вероятности классов при условии каждого индивидуального признака:

```class_probas = classifier.predict_proba(word_vectors)```

Информативность (важность признака) для класса 1 - это вероятность класса 1:

```importances = np.squeeze(class_probs[:, 1:], axis=1)```

In [17]:
setup_experiment(
    "tfidf", "logreg",
    vectorizer_kwargs={
        "tokenizer": mystem_lemmatize,
        "token_pattern": None,
        "stop_words": russian_stopwords,
        "max_df": 0.9,
        "min_df": 0.001,
        "ngram_range": (1, 2)
    },
    classifier_kwargs={
        "C": 0.1,
        "class_weight": "balanced",
    },
    print_most_informative_features=True,
)

              precision    recall  f1-score   support

         0.0     0.8963    0.8898    0.8930       971
         1.0     0.7762    0.7877    0.7819       471

    accuracy                         0.8564      1442
   macro avg     0.8362    0.8387    0.8374      1442
weighted avg     0.8570    0.8564    0.8567      1442

Top 5 toxic words: хохол, русский, тупой, хохлов, дебил


In [18]:
setup_experiment(
    "count", "bayes",
    vectorizer_kwargs={
        "tokenizer": mystem_lemmatize,
        "token_pattern": None,
        "stop_words": russian_stopwords,
        "max_df": 0.9,
        "min_df": 0.001,
        "ngram_range": (1, 1)
    },
    classifier_kwargs={
        "alpha": 10.0,
        "fit_prior": False,
    },
    print_most_informative_features=True,
)

              precision    recall  f1-score   support

         0.0     0.8874    0.8847    0.8860       971
         1.0     0.7637    0.7686    0.7661       471

    accuracy                         0.8467      1442
   macro avg     0.8256    0.8266    0.8261      1442
weighted avg     0.8470    0.8467    0.8469      1442

Top 5 toxic words: хохол, хохлов, дебил, дегенерат, шлюха


In [19]:
setup_experiment(
    "count", "tree",
    vectorizer_kwargs={
        "tokenizer": mystem_lemmatize,
        "token_pattern": None,
        "stop_words": russian_stopwords,
        "max_df": 0.9,
        "min_df": 0.001,
        "ngram_range": (1, 1)
    },
    classifier_kwargs={
        "min_samples_split": 600,
        "criterion": "log_loss",
    },
    print_most_informative_features=True,
)

              precision    recall  f1-score   support

         0.0     0.8685    0.8568    0.8626       971
         1.0     0.7128    0.7325    0.7225       471

    accuracy                         0.8162      1442
   macro avg     0.7906    0.7947    0.7926      1442
weighted avg     0.8176    0.8162    0.8169      1442

Top 5 toxic words: хохол, тупой, русский, хохлов, ебать


In [20]:
setup_experiment(
    "count", "forest",
    vectorizer_kwargs={
        "tokenizer": mystem_lemmatize,
        "token_pattern": None,
        "stop_words": russian_stopwords,
        "max_df": 0.9,
        "min_df": 0.001,
        "ngram_range": (1, 1)
    },
    classifier_kwargs={
        "min_samples_split": 600,
        "criterion": "log_loss",
        "n_estimators": 200,
    },
    print_most_informative_features=True,
)

              precision    recall  f1-score   support

         0.0     0.8673    0.9156    0.8908       971
         1.0     0.8034    0.7113    0.7545       471

    accuracy                         0.8488      1442
   macro avg     0.8353    0.8134    0.8226      1442
weighted avg     0.8464    0.8488    0.8463      1442

Top 5 toxic words: хохол, русский, хохлов, тупой, дебил
